# PostGPT-2
https://www.kaggle.com/baekseungyun/gpt-2-with-huggingface-pytorch<br>

In [ ]:
##Parameter setting
setEpoch = 51
setLearningRate = 0.00002
setEpsilon = 1e-8
setBatch = 16
setMaxLength = 128
setSeed = 42
setTry = 2

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers
!pip install tensorflow
!pip install keras

     |████████████████████████████████| 3.4 MB 9.6 MB/s 
     |████████████████████████████████| 3.3 MB 53.6 MB/s 
     |████████████████████████████████| 61 kB 448 kB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
import torch
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split

from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import set_seed, GPT2LMHeadModel, PreTrainedTokenizerFast, GPT2ForSequenceClassification, GPT2Config
from transformers import AdamW, get_cosine_schedule_with_warmup

import numpy as np
import random
import time
import datetime

In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for currentTry in range(1,setTry):
  postpositions = ["Eyse","Ey","Lo"] 
  labelNumber = 0
  
  for postposition in postpositions:

    if postposition == "Lo":#6
      labelNumber = 6

      def outreault(guess):
        guess = int(guess)
        outFunction = ""
        if guess == 0:
          outFunction = "FNS"
        elif guess == 1:
          outFunction = "INS"
        elif guess == 2:
          outFunction = "DIR"
        elif guess == 3:
          outFunction = "EFF"
        elif guess == 4:
          outFunction = "CRT"
        elif guess == 5:
          outFunction = "LOC"
        return outFunction

    elif postposition == "Eyse":#2
      labelNumber = 2

      def outreault(guess):
        guess = int(guess)
        outFunction = ""
        if guess == 0:
          outFunction = "SRC"
        elif guess == 1:
          outFunction = "LOC"
        return outFunction

    elif postposition == "Ey":#8
      labelNumber = 8

      def outreault(guess):
        guess = int(guess)
        outFunction = ""
        if guess == 0:
          outFunction = "FNS"
        elif guess == 1:
          outFunction = "INS"
        elif guess == 2:
          outFunction = "GOL"
        elif guess == 3:
          outFunction = "EFF"
        elif guess == 4:
          outFunction = "CRT"
        elif guess == 5:
          outFunction = "LOC"
        elif guess == 6:
          outFunction = "AGT"
        elif guess == 7:
          outFunction = "THM"
        return outFunction

    # 1. Model and Tokenizer
    # https://github.com/SKT-AI/KoGPT2
    

    set_seed(setSeed) 

    model_config = GPT2Config.from_pretrained('skt/kogpt2-base-v2', num_labels=labelNumber) 
    model = GPT2ForSequenceClassification.from_pretrained('skt/kogpt2-base-v2', config=model_config)

    tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
      bos_token='</s>', eos_token='</s>', unk_token='<unk>',
      pad_token='<pad>', mask_token='<mask>') 
    tokenizer.padding_side = "left" # Very Important
    tokenizer.pad_token = tokenizer.eos_token

    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = model.config.eos_token_id



    #2. Build Dataset
    testFileDir = fileDir = "drive/My Drive/2022/AdverbialPostpositions/Data/test_"+postposition+".csv"
    testFr = open(testFileDir, 'r')
    testContents = testFr.readlines()
    testFr.close()

    test = pd.DataFrame(columns=('Sentence','Label'))
    i = 0

    for content in testContents:
        if i == 0:
            pass
        else:
            infos = content.split(",")
            label = int(infos[1])
            sentence = infos[2].replace("\n", "")
            test.loc[i] = [sentence, label]
        i = i + 1

    test['Sentence'] = test['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', " ", regex=True)
    test['Sentence'] = test['Sentence'].str.replace(r'\t+', " ", regex=True)
    test['Sentence'] = test['Sentence'].str.replace(r'[\\n]+'," ", regex=True)
    test['Sentence'] = test['Sentence'].str.replace(r'[\s]+', " ", regex=True)
    test['Sentence'] = test['Sentence'].str.strip()

    # 분류 추출
    test_labels = test['Label']
    # 문장 추출
    testSentences = test['Sentence']

    print(len(testSentences))

    fileDir = "drive/My Drive/2022/AdverbialPostpositions/Data/train_"+postposition+".csv"
    fr = open(fileDir, 'r')
    contents= fr.readlines()
    fr.close()

    train = pd.DataFrame(columns=('index', 'Label', 'Sentence'))
    i = 0
    index = ""
    label = ""
    sentence = ""
    for content in contents:
        if i == 0:
            pass
        else:
            infos = content.split(",")
            index = infos[0]
            label = int(infos[1])
            sentence = infos[2].replace("\n","")
            train.loc[i] = [index, label, sentence]
        i = i + 1


    #정제하기

    train['Sentence'] = train['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
    train['Sentence'] = train['Sentence'].str.replace(r'\t+', " ", regex=True)
    train['Sentence'] = train['Sentence'].str.replace(r'[\\n]+'," ", regex=True)
    train['Sentence'] = train['Sentence'].str.replace(r'[\s]+', " ", regex=True)
    train['Sentence'] = train['Sentence'].str.strip()


    class Dataset(Dataset):
        def __init__(self, trainData, testData, checked):
            self.checked = checked

            if self.checked==True:
              self.data = trainData
            else:
              self.data = testData
            # self.data = pd.read_csv(os.path.join("drive/My Drive/2022/AdverbialPostpositions/Data/", "train_"+postposition+".csv" if train else "test_"+postposition+".csv"))
        
        def __len__(self):
            return len(self.data)
        
        def __getitem__(self, index):
            record = self.data.iloc[index]
            text = record['Sentence']
            if self.checked==True:
                return {'Sentence': text, 'label': record['Label']}
            else:
                return {'Sentence': text, 'label': '0'}

    train_dataset = Dataset(train, test, True)
    test_dataset = Dataset(train, test, False)


    # train_dataset = train
    # test_dataset = test

    #3. Data Collator
    class Gpt2ClassificationCollator(object):
        def __init__(self, tokenizer, max_seq_len=None):
            self.tokenizer = tokenizer
            self.max_seq_len = max_seq_len
            
            return
        
        def __call__(self, sequences):
            texts = [sequence['Sentence'] for sequence in sequences]
            labels = [int(sequence['label']) for sequence in sequences]
            inputs = self.tokenizer(text=texts,
                                    return_tensors='pt',
                                    padding=True,
                                    truncation=True,
                                    max_length=self.max_seq_len)
            inputs.update({'labels': torch.tensor(labels)})
            
            return inputs

    gpt2classificationcollator = Gpt2ClassificationCollator(tokenizer=tokenizer,
                                                            max_seq_len=setMaxLength)

    #4. DataLoader

    train_size = int(len(train_dataset) * 0.8)
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=setBatch,
                                  shuffle=True,
                                  collate_fn=gpt2classificationcollator)
    val_dataloader = DataLoader(dataset=val_dataset,
                                batch_size=setBatch,
                                shuffle=False,
                                collate_fn=gpt2classificationcollator)
    test_dataloader = DataLoader(dataset=test_dataset,
                                batch_size=setBatch,
                                shuffle=False,
                                collate_fn=gpt2classificationcollator)

    #5. Optimizer & Lr Scheduler

    total_epochs = setEpoch

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters,
                      lr=setLearningRate,
                      eps=setEpsilon)

    num_train_steps = len(train_dataloader) * total_epochs
    num_warmup_steps = int(num_train_steps * 0.1) 

    lr_scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                  num_warmup_steps=num_warmup_steps,
                                                  num_training_steps = num_train_steps)


    #6. Train & Validation
    import torch

    def train(dataloader, optimizer, scheduler, device_):
        global model
        model.train()
        
        prediction_labels = []
        true_labels = []
        
        total_loss = []
        
        for batch in dataloader:
            true_labels += batch['labels'].numpy().flatten().tolist()
            batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
            
            
            outputs = model(**batch)
            loss, logits = outputs[:2]
            logits = logits.detach().cpu().numpy()
            total_loss.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # prevent exploding gradient

            optimizer.step()
            scheduler.step()
            
            prediction_labels += logits.argmax(axis=-1).flatten().tolist()
        
        return true_labels, prediction_labels, total_loss 

    def validation(dataloader, device_):
        global model
        model.eval()
        
        prediction_labels = []
        true_labels = []

        embedding_outputs = []
        
        total_loss = []

        outputs = []
        
        for batch in dataloader:
            true_labels += batch['labels'].numpy().flatten().tolist()
            batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
            
            with torch.no_grad():
                outputs = model(**batch)
                loss, logits = outputs[:2]
                logits = logits.detach().cpu().numpy()
                total_loss.append(loss.item())

                prediction_labels += logits.argmax(axis=-1).flatten().tolist()

                embedding_outputs += logits.tolist()

                outputs = outputs
            
        return true_labels, prediction_labels, total_loss, outputs, embedding_outputs


    #7. Run

    f = open("drive/My Drive/2022/AdverbialPostpositions/Output/GPT2/"+postposition+"/Outcomes/"+postposition+"_accuracy_trial_"+str(currentTry)+"_epoch_"+str(setEpoch)+".txt", 'w')
    f.write("epoch,sentence,originalLabel,predictedLabel,predictedFunction,result"+"\n")

    from sklearn.metrics import classification_report, accuracy_score

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    all_loss = {'train_loss': [], 'val_loss': []}
    all_acc = {'train_acc': [], 'val_acc': []}
    outputs = []

    for epoch in range(total_epochs):
        y, y_pred, train_loss = train(train_dataloader, optimizer, lr_scheduler, device)
        train_acc = accuracy_score(y, y_pred)
        
        y, y_pred, val_loss, outputs, logits_labels = validation(val_dataloader, device)
        val_acc = accuracy_score(y, y_pred)
        
        all_loss['train_loss'] += train_loss
        all_loss['val_loss'] += val_loss
        
        all_acc['train_acc'].append(train_acc)
        all_acc['val_acc'].append(val_acc)

        outputs = outputs

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch + 1, total_epochs))
        print('Training...')
        
        print(f'Epoch: {epoch}, train_loss: {torch.tensor(train_loss).mean():.3f}, train_acc: {train_acc:.3f}, val_loss: {torch.tensor(val_loss).mean():.3f}, val_acc: {val_acc:.3f}') 

        y, y_pred, val_loss, outputs, logits_labels = validation(test_dataloader, device)

        # print("y",y)
        print("y_pred",y_pred)
        print("y_pred",len(y_pred))
        print("logits_labels",logits_labels)
        print("logits_labels",len(logits_labels))
        print("logits_labels",len(logits_labels[0]))

        sentence_array = []
        for i in range(0,len(test['Sentence'])):
          each_array = []
          for j in range(0,len(logits_labels[i])):
            each_array.append(float(logits_labels[i][j]))
          sentence_array.append(each_array)

        initial_df = pd.DataFrame(sentence_array)

        print("sentence_array",sentence_array)

        # print(len(sentence_array)) # 479
        # print(len(sentence_array[0])) # 37
        
        from sklearn.manifold import TSNE
        tsne = TSNE(n_components=2, random_state=0)
        tsne_obj= tsne.fit_transform(initial_df)

        tsne_df = pd.DataFrame({'X':tsne_obj[:,0],'Y':tsne_obj[:,1],'Label':test_labels})

        tsne_df.to_csv("drive/My Drive/2022/AdverbialPostpositions/Output/GPT2/"+postposition+"/t-SNE/"+postposition+"_tSNE_trial_"+str(currentTry)+"_epoch_"+str(epoch)+".csv")

        import numpy as np   
        import pandas as pd 
        from plotnine import *

        print("")
        print("  Network visualization  ")
        print(ggplot(tsne_df, aes(x='X', y='Y')) + geom_point(aes(colour = 'Label')))


        totalNum = 0
        correctNum = 0
        for each in range(0, len(testSentences)):
            # print(test['Label'][each + 1], test['Sentence'][each + 1])
            # print("y_pred", len(y_pred))
            guess = str(y_pred[each])
            if guess == str(test['Label'][each + 1]):
                # print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(O)")
                f.write(str(epoch+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label'][each + 1]) + "," + guess + "," + outreault(guess)+ ",1" + "\n")
                correctNum = correctNum + 1
            else:
                f.write(str(epoch+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label'][each + 1]) + "," + guess + "," + outreault(guess) + ",0" + "\n")       
                # print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(X)")
            totalNum = totalNum + 1

        print("totalNum: ", totalNum, " correctNum: ", correctNum, " accuracy: ", (correctNum/totalNum))

    f.close()

    print("")
    print("Training complete!")

Output hidden; open in https://colab.research.google.com to view.